In [1]:
import pandas as pd 
import datetime as dt
import os

Functions of this notebook:
* Read raw data downloaded from Canvas (stored in directory ".../Data/raw")
* Combine data by stock
* Outputs one .csv file for every stock (with name "ticker.csv" and stored in directory ".../Data/cleaned")

NOTE: 
* Data at 16:00 is removed due to some abnormal pattern observed in volume data. 
* Different folders contaning different stocks. Only stocks appearing in all folders are included in the final results.

### Directory Setting

In [2]:
dir_working = "/Users/user/Desktop/E4733 AT/Project/Coding Environment/Codes"
os.chdir(dir_working)
dir_data = "../Data/raw"
dir_output = "../Data/cleaned"

### 1. List of All Folders

In [3]:
all_folder = os.listdir(dir_data)
p_ds_store = all_folder.count('.DS_Store') 
if p_ds_store != 0:
    p_ds_store = all_folder.index('.DS_Store')
    del all_folder[p_ds_store]
del(p_ds_store)
all_folder.sort()

### 2. Processing the First Folder 

In [4]:
# list of all files in the first folder
dir_data_t = dir_data + '/' + all_folder[0]
allfile = os.listdir(dir_data_t)
p_ds_store = allfile.count('.DS_Store') 
if p_ds_store != 0:
    p_ds_store = allfile.index('.DS_Store')
    del allfile[p_ds_store]
del(p_ds_store)
allfile.sort()

In [5]:
# initialize lists of tickers and data 
ticker_list = []
data_list = []
for i in range(len(allfile)):
    # get ticker name
    ticker_list.append(allfile[i].split('_')[1].split('.')[0])
    # process the data
    data = pd.read_csv(dir_data_t + '/' + allfile[i], header = None, \
                       names = ['date','time','open','high','low','close',\
                                'volume','splits','earnings','dividend'])
    # construct time index
    data.time = data.time.apply(str)
    data['time_len'] = data.time.apply(len)
    data.loc[data.time_len == 3, 'time_adjust'] = '0'
    data.loc[data.time_len != 3, 'time_adjust'] = ''
    data.time = data.time_adjust + data.time
    data['date_time'] = data.date.apply(str) + ' ' + data.time
    data.time = pd.to_datetime(data.date_time,format = '%Y%m%d %H:%M')
    data = data.set_index('time', drop = True)
    # trading hours 
    data['time'] = data.index.time
    data = data.where((data.time >= dt.time(9,30,0)) & \
                      (data.time <= dt.time(15,59,0)))
    data = data.dropna()
    # remove additional columns 
    data = data.drop(['date','time_len','time_adjust','date_time','time'],\
                     axis=1)
    # fill missing data 
    data.open = data.open.resample('T').last().ffill()
    data.high = data.high.resample('T').last().ffill()
    data.low = data.low.resample('T').last().ffill()
    data.close = data.close.resample('T').last().ffill()
    data.volume = data.volume.resample('T').last().ffill()
    data.splits = data.splits.resample('T').last().ffill()
    data.earnings = data.earnings.resample('T').last().ffill()
    data.dividend = data.dividend.resample('T').last().ffill()
    # append to data_list
    data_list.append(data)

In [6]:
# store values in allfile as allfile0 for future use
allfile0 = allfile 

### 3. Processing Remaining Folders 

In [7]:
for i in range(1, len(all_folder)):
    # list of all files 
    dir_data_t = dir_data + '/' + all_folder[i]
    allfile = os.listdir(dir_data_t)
    p_ds_store = allfile.count('.DS_Store') 
    if p_ds_store != 0:
        p_ds_store = allfile.index('.DS_Store')
        del allfile[p_ds_store]
    del(p_ds_store)
    allfile.sort()
    # find intersection of allfile and allfile0
    allfile_new = list(set(allfile0) & set(allfile))
    allfile_new.sort()
    for j in range(len(allfile0)):
        if allfile0[j] not in allfile_new:
            del ticker_list[j]
            del data_list[j]
    allfile0 = allfile_new # update allfile0
    # process each file
    for j in range(len(allfile)):
        # check if file j in in the list 
        if allfile[j] not in allfile0:
            continue 
        # read and clean data
        p_file = allfile0.index(allfile[j]) # index of ticker
        data = pd.read_csv(dir_data_t + '/' + allfile[j], header = None, \
                           names = ['date','time','open','high','low','close',\
                                    'volume','splits','earnings','dividend'])
        data.time = data.time.apply(str)
        data['time_len'] = data.time.apply(len)
        data.loc[data.time_len == 3, 'time_adjust'] = '0'
        data.loc[data.time_len != 3, 'time_adjust'] = ''
        data.time = data.time_adjust + data.time
        data['date_time'] = data.date.apply(str) + ' ' + data.time
        data.time = pd.to_datetime(data.date_time,format = '%Y%m%d %H:%M')
        data = data.set_index('time', drop = True)
        data['time'] = data.index.time
        data = data.where((data.time >= dt.time(9,30,0)) & \
                          (data.time <= dt.time(15,59,0)))
        data = data.dropna()
        data = data.drop(['date','time_len','time_adjust','date_time','time'],\
                         axis=1)
        # fill missing data 
        data.open = data.open.resample('T').last().ffill()
        data.high = data.high.resample('T').last().ffill()
        data.low = data.low.resample('T').last().ffill()
        data.close = data.close.resample('T').last().ffill()
        data.volume = data.volume.resample('T').last().ffill()
        data.splits = data.splits.resample('T').last().ffill()
        data.earnings = data.earnings.resample('T').last().ffill()
        data.dividend = data.dividend.resample('T').last().ffill()
        # append the data to corresponding dataframe 
        data_list[p_file] = pd.concat([data_list[p_file],data])

### 4. Output as .csv Files

In [8]:
for i in range(len(ticker_list)):
    file_name = dir_output + '/' + ticker_list[i] + '.csv'
    data_list[i].to_csv(file_name)